In [ ]:
import sys
import os
import ase
from ase.visualize import view

import matplotlib.pyplot as plt
import numpy as np

sys.path.append('/Users/austin/Documents/GitHub/DataGenSTEM/DataGenSTEM')
import data_generator as dg

%matplotlib ipympl

In [ ]:
atoms = ase.io.read('atoms.cif')

positions = atoms.get_positions()
min_pos = np.min(positions, axis=0)
max_pos = np.max(positions, axis=0)

cell_lengths = max_pos - min_pos
atoms.set_cell(cell_lengths)
atoms.wrap()

xtal = atoms.copy()

# Define the image size in Angstroms
positions = xtal.get_positions()[:, :2]
xmin, xmax = np.min(positions[:, 0]), np.max(positions[:, 0])
ymin, ymax = np.min(positions[:, 1]), np.max(positions[:, 1])
borders = 1
axis_extent = (xmin - borders, xmax + borders, ymin - borders, ymax + borders)

pixel_size = 0.106 # Angstrom/pixel, determines number of points, aka resolution of maps.  the xtal determines the fov

In [ ]:
atom_var = 0.014
potential = dg.get_pseudo_potential(xtal = xtal, pixel_size = pixel_size, sigma = atom_var, axis_extent = axis_extent)

plt.figure()
plt.imshow(potential, cmap='gray', extent=axis_extent)
plt.colorbar()

In [ ]:
# Make point spread function
airy_disk_size = 1
psf = dg.get_point_spread_function(airy_disk_radius = airy_disk_size, size = 32)
psf_resize = dg.resize_image(np.array(psf), n = max(potential.shape)) # for plotting on same axes as image
perfect_image = dg.convolve_kernel(potential, psf)

plt.figure()
plt.imshow(perfect_image, cmap='gray', extent=axis_extent)
plt.colorbar()

In [ ]:
shot_noise = 0.8
noisy_image = dg.add_poisson_noise(perfect_image, shot_noise = shot_noise)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(perfect_image, cmap='gray')
ax[0].set_title('Perfect Image')

ax[1].imshow(noisy_image, cmap='gray')
ax[1].set_title('Noisy Image')
